#모델의 성능 평가를 위한 훈련 세트와 테스트 세트

일반화 성능 - 훈련된 모델의 실전 성능
과도하게 낙관적으로 일반화 성능을 추정한다 : 모델을 이미 학습시킨 훈련 데이터 세트로 모델의 성능을 평가한다. 
    --> 긍정적인 결과가 나올 수 밖에 없음
훈련 데이터 세트를 두 덩이로 나누어 하나는 훈련에 (훈련 세트), 하나는 테스트에 (테스트 세트)나누어 사용한다. 

<훈련 데이터 세트를 훈련 세트와 테스트 세트로 나누는 규칙>
1. 훈련 데이터 세트를 나눌 때는 테스트 세트보다 훈련 세트가 더 많아야 한다.
2. 훈련 데이터 세트를 나누기 전에 양성, 음성 클래스가 훈련 세트나 테스트 세트의 어느 한쪽에 몰리지 않도록 섞어야한다. 


#훈련 세트와 테스트 세트로 나누기

train_test_split()함수 --> 훈련 세트 75%, 테스트 세트 25%의 비율로 나누어준다. 

In [6]:
from sklearn.model_selection import train_test_split#train_test_split함수를 사용하겠다. 
import numpy as np
from sklearn.datasets import load_breast_cancer #사이킷런에서 제공하는 datasets의 모듈에서 유방암 데이터를 불러온다. 
cancer = load_breast_cancer() #데이터를 불러온다
np.unique(cancer.target, return_counts = True)
x = cancer.data #x에는 데이터
y = cancer.target #y에는 타겟데이터

x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y, test_size=0.2, random_state=42)
# stratify = y : 이 함수는 기본적으로 데이터를 나누기 전에 섞지만 일부 클래스 비율이 불균형한 경우에는 stratify를 y로 지정한다.
#test_size = 0.2 : 원래는 75%, 25%의 비율이지만, 비율을 조정하기 위해서 사용한다. 즉 20%의 테스트 세트로 나누기 위해서는 0.2라고 지정해주면 된다. 
#random_state = 42 : 무작위로 데이터 세트를 섞고, 나눈다. 섞은 다음 나눈 결과가 항상 일정하도록 매개변수에 난수 값을 지정한다. 

print(x_train.shape, x_test.shape)
#지정해준 비율과 같이 4:1로 잘 나누어졌다는 것을 출력물로 보여진다.
np.unique(y_train, return_counts=True)
#y의 훈련세트를 본결과 양성 클래스와 음성클래스의 비율이 그대로 유지한다. 

(455, 30) (114, 30)


(array([0, 1]), array([170, 285], dtype=int64))

#로지스틱 회귀 구현하기

<구현사항>
1. 정방향 계산 : 정방향으로 데이터가 흘러가는 과정
2. 역방향 계산 : 가중치를 업데이트하기 위해 역방향으로 데이터가 흘러가는 과정

In [13]:
class LogisticNeuron:
    def __init__(self):
        self.w = None
        self.b = None #가중치와 절편을 초기화 시킨다. 
    def forpass(self,x):
        z = np.sum(x*self.w)+ self.b 
        return z
    def backprop(self, x, err): #그래디언트를 계산한다. 
        w_grad = x * err
        b_grad = 1 * err
        return w_grad, b_grad
    #훈련하는 메서드 구현하기

    def activation(self, z):
        a = 1/(1+np.exp(-z)) #시그모이드 계산 이때, exp는 넘파이에서의 자연상수의 지수 함수를 계산하는 함수이다. 
        return a
    def fit(self, x, y, epochs = 100):#100번 반복
        self.w = np.ones(x.shape[1]) # 가중치 초기화 1로
        self.b = 0 
        for i in range(epochs):
            for x_i, y_i in zip(x,y):
                z = self.forpass(x_i) #정방향 계산
                a = self.activation(z) #활성화 함수는 activation이라는 함수로 진행한다.
                err = -(y_i -a) #오차 계산
                w_grad, b_grad = self.backprop(x_i, err) #역방향 계산
                self.w -= w_grad
                self.b -= b_grad # 업데이트 
    def predict(self, x):
        z = [self.forpass(x_i) for x_i in x] #선형 함수 적용
        a = self.activation(np.array(z)) # 활성화 함수 적용
        return a>0.5 #임계함수 적용
    
# 모델 훈련하기 
from sklearn.model_selection import train_test_split#train_test_split함수를 사용하겠다. 
import numpy as np
from sklearn.datasets import load_breast_cancer #사이킷런에서 제공하는 datasets의 모듈에서 유방암 데이터를 불러온다. 
cancer = load_breast_cancer() #데이터를 불러온다
np.unique(cancer.target, return_counts = True)
x = cancer.data #x에는 데이터
y = cancer.target #y에는 타겟데이터
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y, test_size=0.2, random_state=42)

neuron = LogisticNeuron()
neuron.fit(x_train, y_train) #모델 훈련하기 

np.mean(neuron.predict(x_test) == y_test) #예측한 0, 1 등의 결과값들의 예측값을 평균내어 정확도를 측정
    

c:\users\dkstj\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


0.8245614035087719


82%의 정확도를 보인다. 
